In [1]:
import glob
import numpy as np
csv_files = (glob.glob("C:/Users/Vishnu Kumar/Documents/IPL/raw/*info.csv")) 

In [2]:
import pandas as pd


df_append = pd.DataFrame()

for file in csv_files:
            df_temp = pd.read_csv(file,sep='漢ழ்字', engine='python',header=None)
            df_temp['ID'] = file
            df_temp = df_temp[(df_temp[0].str.startswith('info,toss_winner')) | (df_temp[0].str.startswith('info,toss_decision'))]
            df_append = pd.concat([df_append,df_temp])

df_append = df_append.reset_index()
df_append[['info', 'attrib','value']] = df_append[0].str.split(",", expand = True)
df_append = df_append.drop(columns=['index',0,'info'])
df_append['ID'] = df_append['ID'].str[40:-9]
toss_winner = df_append[df_append['attrib']=='toss_winner'].copy()
toss_decision = df_append[df_append['attrib']=='toss_decision'].copy()
toss_winner['ID'] = toss_winner['ID'].astype('int')
toss_decision['ID'] = toss_decision['ID'].astype('int')

In [49]:
all_matches = pd.read_csv("C:/Users/Vishnu Kumar/Documents/IPL/raw/all_matches.csv")
# all_matches
all_matches = pd.merge(all_matches, toss_winner, how="left" ,left_on = "match_id", right_on = "ID" )
all_matches = all_matches.drop(columns=["ID", "attrib"])
all_matches = all_matches.rename(columns={"value": "toss_winner"})
all_matches = pd.merge(all_matches, toss_decision, how="left" ,left_on = "match_id", right_on = "ID" )
all_matches = all_matches.drop(columns=["ID", "attrib"])
all_matches = all_matches.rename(columns={"value": "toss_decision"})
all_matches = all_matches[all_matches['innings']<=2]
all_matches['season'] = all_matches['season'].replace(['2007/08', '2009', '2009/10', '2011' ,'2012','2018', '2019', '2020/21', '2021'], [2008,2009,2010,2011,2012,2018,2019,2020,2021])
all_matches['toss_win_batting'] = all_matches.apply(lambda x: 1 if x['toss_winner'] == x['batting_team'] else 0, axis=1)
all_matches['toss_decision'] = all_matches.apply(lambda x: 1 if x['toss_decision'] == 'bat' else 0, axis=1)
all_matches['player_dismissed'] = all_matches.apply(lambda x: 1 if x['player_dismissed'] == x['striker'] else 0, axis=1)
all_data = all_matches.copy()
# all_matches = pd.concat([all_matches, toss_winner], axis=1, join='outer', join_axes=[match_id,ID])

C:\Users\Vishnu Kumar\AppData\Local\Temp\ipykernel_15892\579395103.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  all_matches = pd.read_csv("C:/Users/Vishnu Kumar/Documents/IPL/raw/all_matches.csv")


In [50]:
all_matches['ball'] = (all_matches['ball'].astype('int')*6)+((all_matches['ball'] - all_matches['ball'].astype('int'))*10)

In [51]:
all_matches['wicket'] = all_matches['wicket_type'].notnull().astype(int)

In [52]:
all_matches = all_matches.fillna(0)

In [53]:
all_matches['byes'] = all_matches['byes'] + all_matches['legbyes']
all_matches['extras'] = all_matches['extras'] - all_matches['byes']

In [54]:
all_matches = all_matches.drop(columns=['match_id','season','start_date','innings','batting_team','non_striker','bowling_team','player_dismissed','other_wicket_type','toss_winner','toss_decision','toss_win_batting','wides','noballs','legbyes','penalty','other_player_dismissed','wicket_type'])

In [55]:
all_matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225793 entries, 0 to 225953
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   venue         225793 non-null  object 
 1   ball          225793 non-null  float64
 2   striker       225793 non-null  object 
 3   bowler        225793 non-null  object 
 4   runs_off_bat  225793 non-null  int64  
 5   extras        225793 non-null  float64
 6   byes          225793 non-null  float64
 7   wicket        225793 non-null  int32  
dtypes: float64(3), int32(1), int64(1), object(3)
memory usage: 14.6+ MB


In [61]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten

# load the data
# all_matches = pd.read_csv('path/to/all_matches.csv')

# select relevant columns
data = all_matches[['venue', 'striker', 'bowler', 'ball', 'runs_off_bat', 'extras', 'byes', 'wicket']]

# one-hot encode categorical features
enc = OneHotEncoder()
venue_enc = enc.fit_transform(data['venue'].values.reshape(-1, 1))
striker_enc = enc.fit_transform(data['striker'].values.reshape(-1, 1))
bowler_enc = enc.fit_transform(data['bowler'].values.reshape(-1, 1))
ball_enc = enc.fit_transform(data['ball'].values.reshape(-1, 1))
X = pd.concat([venue_enc.toarray(), striker_enc.toarray(), bowler_enc.toarray(), ball_enc.toarray()], axis=1)

# normalize numerical features
num_cols = ['runs_off_bat', 'extras', 'byes', 'wicket']
X[num_cols] = (data[num_cols] - data[num_cols].mean()) / data[num_cols].std()

# define the model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(X.values.reshape((X.shape[0], X.shape[1], 1)), data[num_cols].values, epochs=10, validation_split=0.2)

# evaluate the model
model.evaluate(X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1)), y_test)


TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [48]:
import numpy as np

# Use the model to make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the mean squared error (MSE)
mse = np.mean(np.square(y_test - y_pred))
print('MSE: {}'.format(mse))

# Calculate the root mean squared error (RMSE)
rmse = np.sqrt(mse)
print('RMSE: {}'.format(rmse))

# Calculate the R-squared
ss_res = np.sum(np.square(y_test - y_pred))
ss_tot = np.sum(np.square(y_test - np.mean(y_test)))
r_squared = 1 - (ss_res / ss_tot)
print('R-squared: {}'.format(r_squared))


1412/1412 [==============================] - 3s 2ms/step
MSE: runs_off_bat    2.582900
extras          0.063515
byes            0.054334
wicket          0.047245
dtype: float64
RMSE: runs_off_bat    1.607140
extras          0.252021
byes            0.233095
wicket          0.217360
dtype: float64
R-squared: runs_off_bat    0.008250
extras          0.000308
byes           -0.000153
wicket          0.005384
dtype: float64


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [21]:
import sklearn.metrics as metrics

# predict on test data
y_pred = model.predict(X_test_scaled)

# evaluate the model
test_loss = model.evaluate(X_test_scaled, y_test, verbose=0)
accuracy = model.accuracy(y_test, y_pred.round())

print(f"Loss: {test_loss:.4f}, Accuracy: {100:.2f}%")


1412/1412 [==============================] - 3s 2ms/step


AttributeError: 'Sequential' object has no attribute 'accuracy'

In [24]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
accuracy_score(y_test, model.predict(X_test_scaled))

1412/1412 [==============================] - 3s 2ms/step


ValueError: Classification metrics can't handle a mix of multiclass-multioutput and continuous-multioutput targets

In [12]:
print(y_test.shape)
print(np.unique(y_test))


(45159, 14)
[0.         0.14285714 0.16666667 0.2        0.28571429 0.33333333
 0.4        0.42857143 0.5        0.57142857 0.6        0.66666667
 0.71428571 0.8        0.83333333 1.        ]


In [32]:
pd.DataFrame(y_pred).to_csv('y_pred.csv')
pd.DataFrame(y_test).to_csv('y_test.csv')


In [27]:
test_set.iloc[:, 0:4]

,runs_off_bat,extras,byes,wicket
5,0,0.0,0.0,0
10,6,0.0,0.0,0
15,0,0.0,1.0,0
16,4,0.0,0.0,0
18,0,0.0,0.0,0
...,...,...,...,...
225929,1,0.0,0.0,0
225930,0,0.0,0.0,0
225942,0,0.0,0.0,0
225947,0,0.0,0.0,0


In [35]:
print(all_matches)
all_matches['venue'] = label_encoder.fit_transform(all_matches['venue'])


None


TypeError: 'NoneType' object is not subscriptable

In [37]:
print(all_matches)

None
